# Multi-tenant Chat with Papers - Query papers
## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

USERNAME = os.getenv("USERNAME")
PAPERS_NAME = f"{USERNAME}_papers"

print(PAPERS_NAME)

## Connect to Weaviate

In [ ]:
import weaviate

client = weaviate.connect_to_local(
    headers={
        "X-OpenAI-Api-Key": OPENAI_API_KEY,
    },
)

client.is_ready()

## Vector search on tenants

In [ ]:
papers = client.collections.get(PAPERS_NAME)

ten = papers.with_tenant("2212-10496")

response = ten.query.near_text(
    query="Unsupervised learning",
    limit=5,
)

for item in response.objects:
    print(item.properties["chunk"])

## Genartive Search with tenants

In [ ]:
# Let's add some colour to our lives :)
BLUE   = "\033[94m"
PURPLE = "\033[95m"
RESET  = "\033[0"

In [ ]:
papers = client.collections.get(PAPERS_NAME)

ten2212 = papers.with_tenant("2212-10496")

response = ten2212.generate.near_text(
    query="Unsupervised learning",
    limit=5,
    single_prompt="What does the following text describe: {chunk}",
)

for item in response.objects:
    print(f"{BLUE}=== Chunk ===")
    print(item.properties["chunk"])
    
    print(f"{PURPLE}=== Generated Response ===")
    print(item.generative.text, "\n")

In [ ]:
papers = client.collections.get(PAPERS_NAME)

ten2212 = papers.with_tenant("2212-10496")

response = ten2212.generate.near_text(
    query="Unsupervised learning",
    limit=5,
    grouped_task="Explain how unsupervised learning works. Use only the provided content.",
    grouped_properties=["chunk"]
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Chunks ===")
for item in response.objects:
    print(item.properties["chunk"])

## End-to-end

In [ ]:
def paper_rag(paper_id, query, prompt):
    papers = client.collections.get(PAPERS_NAME)
    ten = papers.with_tenant(paper_id)

    response = ten.generate.near_text(
        query=query,
        limit=5,
        grouped_task=prompt + " Use only the provided content.",
        grouped_properties=["chunk"],
    )

    return {
        "title": response.objects[0].properties["title"],
        "source": [p.properties["chunk"] for p in response.objects],
        "generated": response.generative.text
    }

In [ ]:
paper_rag(
    "2212-10496",
    "Unsupervised learning",
    "Explain how unsupervised learning works"
)

## List tenants

In [ ]:
papers = client.collections.get(PAPERS_NAME)
papers.tenants.get()

## Close the client

In [ ]:
client.close()